In [ ]:
import serial
from time import sleep
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt 
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation

from ipywidgets import interact

In [ ]:
ser = serial.Serial('/dev/cu.wchusbserial1410', 9600) # Establish the connection on a specific port
print(ser.readline())

In [ ]:
def servo(angles):
    s = b'<s'
    for a in angles:
        if a > 1:
            a = 1
        elif a < -1:
            a = -1
        a = int(a * 150 + 150)
        
        s += b'%03d' % a
    s += b'>'
    ser.write(s)

In [ ]:
def getMotion():
    ser.write(b'<m>')
    s = ser.readline()
    s = s.decode("utf-8")
    s = s.replace('\r', '').replace('\n','')
    return [float(a) for a in s.split('\t')]

In [ ]:
# plot motion
m = []
for _ in range(1000):
    clear_output(wait=True)
    m.append(np.array(getMotion())[:3])
    a = np.array(m)
    fig = plt.plot(a[:,0], c='r') # front and back
    fig = plt.plot(a[:,1], c='b') # left and right
    fig = plt.plot(a[:,2], c='g')
    plt.show()

In [ ]:
def height_rot_angles(h,r):
    return [-h,r-.2,h-.1,r,h,r,-h+.1,r]

def height_rot(h,r):
    servo(height_rot_angles(h,r))
    
height_rot(1,0)

In [ ]:
height_rot(0,0)

In [ ]:
# sleep!

height_rot(0,0)
sleep(.5)
height_rot(1,0)
sleep(.2)
height_rot(1,.2)
sleep(.2)
height_rot(1,-.2)
sleep(.2)
height_rot(1,0)

In [ ]:
# turn 

for t in np.linspace(0, np.pi*2, 60):
    d = -1
    height_rot(np.cos(d*t),np.sin(d*t))
    sleep(.01)

In [ ]:
# sense motion

height_rot(.8,0)

last_z = getMotion()[0]
while True:
    z = getMotion()[0]
    clear_output(wait=True)
    diff = abs(z-last_z)
    print(diff)
    if diff > 200:
        for t in np.linspace(0,np.pi*2,10):
            height_rot(.8,np.sin(t)*.5)
            sleep(.05)
        for t in range(10):
            z = getMotion()[0]
            last_z = z
            sleep(.1)
    last_z = z
    sleep(.1)

In [ ]:
# move forward

def forward(step=20):
    s = height_rot_angles(.7,0)
    servo(s)
    for t in np.linspace(0, 2*np.pi, step):
        st = s.copy()
        a = np.sin(t)*.5
        b = np.cos(-t)*.3 - .3

        st[0] += a
        st[1] += b

        st[4] -= a
        st[5] -= b

        servo(st)
        sleep(.01)

    for t in np.linspace(0, 2*np.pi, step):
        st = s.copy()
        a = np.sin(t)*.3
        b = np.cos(-t)*.3 - .3

        st[6] += a
        st[7] -= b

        st[2] -= a
        st[3] += b

        servo(st)
        sleep(.01)
        
forward()

In [ ]:
# move backward

def backward(step=20):
    s = height_rot_angles(.7,0)
    servo(s)
    for t in np.linspace(2*np.pi, 0, step):
        st = s.copy()
        a = np.sin(t)*.5
        b = np.cos(-t)*.3 - .3

        st[0] += a
        st[1] += b

        st[4] -= a
        st[5] -= b

        servo(st)
        sleep(.01)

    for t in np.linspace(2*np.pi, 0, step):
        st = s.copy()
        a = np.sin(t)*.3
        b = np.cos(-t)*.3 - .3

        st[6] += a
        st[7] -= b

        st[2] -= a
        st[3] += b

        servo(st)
        sleep(.01)
        
backward()

In [ ]:
forward()
backward()

In [ ]:
# clear data

angles = [] # joint angles
senses = [] # motion sensor

In [ ]:
# data collection

s = np.array([-1,0,1,0,1,0,-1,0], dtype=np.float64) # base position
last_s = s.copy()
servo(s)
sleep(1)

while True:
    # move randomly
    rd = np.random.randn(8) * .3
    #rd[[1,3,5,7]] = 0 # limit to only outer joints
    test_angles = np.clip(s+rd, -1, 1)
    
    for i in np.linspace(0, 1, 10):
        servo((1-i) * last_s + i * test_angles)
    
    last_s = test_angles.copy()
    sleep(.5)
    
    # sample some motions
    sense_sum = np.zeros(3)
    for i in range(10):
        sense_sum += np.array(getMotion())[:3]
        sleep(.05)
        
    sense = sense_sum/10.0
    
    # save experience pairs
    angles.append(test_angles.copy())
    senses.append(sense)
    
    clear_output(wait=True)
    a = np.array(senses)
    fig = plt.plot(a[:,0], c='r') # front and back
    fig = plt.plot(a[:,1], c='b') # left and right
    #fig = plt.plot(a[:,2], c='g') # up and down
    plt.show()

In [ ]:
# normalize motion

axis = 0

u = np.mean(senses, axis=0)
std = np.std(senses, axis=0)

senses_norm = (senses - u) / std

single_axis = np.array(senses_norm)[:,axis]
fig = plt.plot(single_axis)

In [ ]:
def line_fit(single_axis, joint_states):
    m, b = np.polyfit(single_axis, joint_states, 1)
    
    T = np.linspace(np.min(single_axis), np.max(single_axis), 100)

    fig = plt.scatter(single_axis, joint_states)
    fig = plt.plot(T, [m*t + b for t in T])
    
    return m, b

In [ ]:
line_fit(single_axis, np.array(angles)[:,0])

In [ ]:
coeff = [line_fit(single_axis, np.array(angles)[:,j]) for j in range(8)]

In [ ]:
T = np.linspace(np.min(single_axis)-2, np.max(single_axis)+2, 20)

for desired_h in T:
    js = np.array([c[0]*desired_h + c[1] for c in coeff])
    servo(js)
    sleep(.1)

In [ ]:
# multi input

model = Sequential()
model.add(Dense(64, input_dim=2)) # two for xy
model.add(Activation('tanh'))
model.add(Dense(8))
model.add(Activation('tanh'))

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

In [ ]:
# model training

axis_vis = senses_norm[:,0]

T = np.linspace(np.min(axis_vis), np.max(axis_vis), 30)

for i in range(20):
    model.fit(senses_norm[:,:2], np.array(angles), verbose=False, epochs=20)
    
    joint_angles = []
    for m in T:
        joint_angles.append(model.predict(np.array([[m, 0]]))[0])
    
    clear_output(wait=True)
    fig = plt.plot(T, joint_angles)
    plt.show()

In [ ]:
def tilt(x, y):
    joint_angles = model.predict(np.array([[x, y]]))[0]
    servo(joint_angles)
    sleep(.1)
    
interact(tilt, x=(-4, 4, .1),
                y=(-4, 4, .1))

In [ ]:
# circle

T = np.linspace(0, np.pi * 2.0, 50)

for t in T:
    joint_angles = model.predict(np.array([[np.sin(t)*3, np.cos(t)*3]]))[0]
    servo(joint_angles)
    sleep(.1)

In [ ]:
# side to side
T = np.linspace(0, np.pi * 2.0, 30)

for t in T:
    joint_angles = model.predict(np.array([[0, np.sin(2*t)*3]]))[0]
    servo(joint_angles)
    sleep(.1)

In [ ]:
# front and back

T = np.linspace(0, np.pi * 2.0, 30)

for t in T:
    joint_angles = model.predict(np.array([[np.sin(2*t)*3, 0]]))[0]
    servo(joint_angles)
    sleep(.1)

In [ ]:
# balance

x = 0
y = 0

while True:fina
    m = np.array(getMotion())
    
    x *= 0.8
    y *= 0.8
    
    x -= m[0]/4000
    y -= m[1]/4000
        
    joint_angles = model.predict(np.array([[x, y]]))[0]
    servo(joint_angles)
    sleep(.05)

In [ ]:
#model.save('good_balance')

In [ ]:
model.load_weights('good_balance')

In [ ]:
"""

# multi input

model = Sequential()
model.add(Dense(64, input_dim=2)) # two for xy
model.add(Activation('tanh'))
model.add(Dense(8))
model.add(Activation('tanh'))

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')
              
"""
